# Julia Comparison

Julia is also often cited as a speedy alternative to Python and Numba. 
Unlike Python, jit-compilation is built into the language, so there is no need to use a separate library.
It is often easier to design large-scale projects in Julia than in numba thanks to this deeper integration of jit-compilation into the language.

Here we will compare the performance of Julia to Python and Numba for the same problem as in our previous examples.

## `integrate_context`
The function `integrate_context` uses a hidden function `calc_rho` to integrate contextual input into a context vector. 
Here we implement it using Julia.

In [1]:
using LinearAlgebra

function calc_rho(cdot, B)
    rho = sqrt(1 + (B * B) * ((cdot * cdot) - 1)) - (B * cdot)
    return rho
end

function integrate_context(c, c_in, B, c_ind)
    cdot = 0
    for i in c_ind[1]:c_ind[2]
        cdot += c[i] * c_in[i]
    end
    rho = calc_rho(cdot, B)
    
    for i in c_ind[1]:c_ind[2]
        c[i] = rho * c[i] + B * c_in[i]
    end
end


integrate_context (generic function with 1 method)

In [2]:
c1 = [0.0, 0.09128709, 0.18257419, 0.27386128, 0.36514837, 0.8660254]
c2 = [0.15655607, 0.24875946, 0.34096284, 0.43316622, 0.52536961, 0.57767384]
B = 0.5
c_ind = [1, 6]  # Julia arrays are 1-indexed

integrate_context(c1, c2, B, c_ind)

In [3]:
using BenchmarkTools

@benchmark integrate_context(c1, c2, B, c_ind)

BenchmarkTools.Trial: 10000 samples with 991 evaluations.
 Range (min … max):  41.574 ns …  1.203 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     42.079 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   46.847 ns ± 18.320 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▄  ▁▂▁  ▃           ▁▁  ▁                                  ▁
  ███████▇██▇█▆▆▆▅▆▆█▇▇██████▇█▇▇▆▆▆▆▅▅▆▅▅▅▅▄▅▅▃▅▆▇▅▃▂▄▄▄▂▄▄▅ █
  41.6 ns      Histogram: log(frequency) by time        95 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In our evaluation of the performance of `integrate_context` as implemented using numba, we found an average runtime of 556 ns +/- 14.2 ns per loop. 

Here, using Julia, we find an average runtime of 46.8 ns +/- 18.3 ns per loop.

This is over 10 times faster than the numba implementation.